In [1]:
pip install transformers datasets accelerate sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install transformers datasets sacremoses

In [8]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import pandas as pd

In [10]:
df = pd.read_csv("hindi_english_parallel.csv")  # Make sure file path is correct
dataset = Dataset.from_pandas(df)
df.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [7]:
split_dataset = dataset.train_test_split(test_size=0.2)

In [9]:
from transformers import MarianTokenizer, MarianMTModel
model_name = "Helsinki-NLP/opus-mt-hi-en"  # For Hindi to English translation
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

C:\Users\ACER\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
force_download = True

In [5]:
!pip install sacremoses

   ---------------------------------------- 0.0/897.5 kB ? eta -:--:--
   ----------------------------------- ---- 786.4/897.5 kB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 897.5/897.5 kB 6.8 MB/s eta 0:00:00


In [13]:
print(dataset.column_names)

['hindi', 'english']


In [15]:
print(dataset)
print(type(dataset))
print(dataset.column_names)
print(dataset[0])

Dataset({
    features: ['hindi', 'english'],
    num_rows: 1561841
})
<class 'datasets.arrow_dataset.Dataset'>
['hindi', 'english']
{'hindi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें', 'english': 'Give your application an accessibility workout'}


In [12]:
def is_valid(example):
    return isinstance(example["hindi"], str) and isinstance(example["english"], str)

clean_dataset = dataset.filter(is_valid)

Filter:   0%|          | 0/1561841 [00:00<?, ? examples/s]

In [14]:
def preprocess_function(examples):
    return tokenizer(
        examples["hindi"],
        text_target=examples["english"],
        max_length=128,
        padding="max_length",
        truncation=True
    )

In [16]:
tokenized_dataset = clean_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1555574 [00:00<?, ? examples/s]

In [18]:
from datasets import DatasetDict

# 90% train, 10% validation split
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)

# Rename keys to match expected names
split_dataset = DatasetDict({
    "train": split_dataset["train"],
    "validation": split_dataset["test"]
})

In [34]:
!pip install --upgrade accelerate

In [18]:
!pip install transformers==4.38.2 accelerate==0.27.2

   ---------------------------------------- 0.0/8.5 MB ? eta -:--:--
   ------ --------------------------------- 1.3/8.5 MB 7.5 MB/s eta 0:00:01
   ------- -------------------------------- 1.6/8.5 MB 8.4 MB/s eta 0:00:01
   ----------- ---------------------------- 2.4/8.5 MB 3.9 MB/s eta 0:00:02
   ---------------------- ----------------- 4.7/8.5 MB 5.8 MB/s eta 0:00:01
   ------------------------------------ --- 7.9/8.5 MB 7.8 MB/s eta 0:00:01
   ---------------------------------------- 8.5/8.5 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.6.0
    Uninstalling accelerate-1.6.0:
      Successfully uninstalled accelerate-1.6.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2


In [20]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

# Optional: Use smaller subsets for faster training
small_train_dataset = split_dataset["train"].select(range(500))        # You can adjust this
small_eval_dataset = split_dataset["validation"].select(range(100))

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# ✅ Training arguments (THIS is where we control epochs & steps)
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,                       # ⬅️ Only 3 epochs!
    logging_steps=100,
    save_total_limit=1,
    save_strategy="no",                       # ⬅️ Don't save checkpoints
    evaluation_strategy="no",                 # ⬅️ Skip evaluation during training
                                    # ⬅️ Optional if on GPU
    report_to="none",                         # ⬅️ Disable W&B/TensorBoard
    predict_with_generate=True,
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train!
trainer.train()

Step,Training Loss
100,0.399300


TrainOutput(global_step=189, training_loss=0.3596232984431837, metrics={'train_runtime': 574.9238, 'train_samples_per_second': 2.609, 'train_steps_per_second': 0.329, 'total_flos': 50847547392000.0, 'train_loss': 0.3596232984431837, 'epoch': 3.0})

In [22]:
trainer.save_model("./my_finetuned_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[61126]], 'forced_eos_token_id': 0}


In [24]:
tokenizer.save_pretrained("./my_finetuned_model")

('./my_finetuned_model\\tokenizer_config.json',
 './my_finetuned_model\\special_tokens_map.json',
 './my_finetuned_model\\vocab.json',
 './my_finetuned_model\\added_tokens.json')

In [4]:
# only for loading

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("./my_finetuned_model")
tokenizer = AutoTokenizer.from_pretrained("./my_finetuned_model")

In [33]:
input_text = "आपका नाम क्या है?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# Generate translation
output_ids = model.generate(**inputs)
translated = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Translated:", translated)

Translated: What is your name?


In [30]:
input_text = "यह एक परीक्षा है"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# Generate translation
output_ids = model.generate(**inputs)
translated = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Translated:", translated)

Translated: It is a test


In [32]:
input_text = "मैं स्कूल जा रहा हूँ।"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# Generate translation
output_ids = model.generate(**inputs)
translated = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Translated:", translated)

Translated: I am going to school.


In [34]:
!pip install evaluate


In [38]:
!pip install sacrebleu jiwer

!pip install rouge_score absl-py nltk


   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------------------- -------------- 1.0/1.6 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 6.2 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24972 sha256=52a68517f9c51b1f8f4fc6f179e87b9fca16a80e61926ae3064dfa0c78402bb9
  Stored in directory: c:\users\acer\appdata\local\pip\cache\wheels\85\9d\af\01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...


In [50]:
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Limit for quick test
validation_data = split_dataset["validation"].select(range(100))
batch_size = 32

predictions = []
references = []

for i in tqdm(range(0, len(validation_data), batch_size)):
    batch = validation_data.select(range(i, min(i + batch_size, len(validation_data))))
    batch_dict = batch.to_dict()

    hindi_texts = batch_dict["hindi"]
    english_refs = batch_dict["english"]

    inputs = tokenizer(hindi_texts, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    outputs = model.generate(**inputs)
    translated_batch = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    predictions.extend(translated_batch)
    references.extend(english_refs)


100%|██████████| 4/4 [03:30<00:00, 52.57s/it]


In [56]:
bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
rouge_score = rouge.compute(predictions=predictions, references=references)
meteor_score = meteor.compute(predictions=predictions, references=references)

print("BLEU:", bleu_score)
print("ROUGE:", rouge_score)
print("METEOR:", meteor_score)



BLEU: {'bleu': 0.1889078247813802, 'precisions': [0.47947761194029853, 0.23143236074270557, 0.1358198451794511, 0.08792846497764531], 'brevity_penalty': 0.9900990911851015, 'length_ratio': 0.9901477832512315, 'translation_length': 1608, 'reference_length': 1624}
ROUGE: {'rouge1': 0.4020023626044187, 'rouge2': 0.21414160421906642, 'rougeL': 0.370264331636888, 'rougeLsum': 0.3707874567552187}
METEOR: {'meteor': 0.35038725039883917}
